In [2]:
from pyscf import gto, scf, ao2mo, fci
import itertools
from copy import copy
import CC, Deprecated.Wick as Wick, GeneralisedWick
import numpy as np
from math import factorial, sqrt

In [3]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [4]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    if isinstance(H, Wick.operatorProduct) or isinstance(H, Wick.operatorSum):
        result.checkNilpotency()
    return result

In [5]:
def projectionManifold(excitationLevel):
    return GeneralisedWick.Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [6]:
h1Tensor = GeneralisedWick.Tensor("f", ['g'], ['g'])
h2Tensor = GeneralisedWick.Tensor("v", ['g', 'g'], ['g', 'g'])
t2Tensor = GeneralisedWick.Tensor("{t_{2}}", ['p', 'p'], ['h', 'h'])

In [7]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

Generalised Wick

In [11]:
h1Tensor.array = h1
h2Tensor.array = ao2mo.kernel(mol, mf.mo_coeff, compact=False).reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)
#h2Tensor.array = 2 * h2Array

In [12]:
h2Tensor.array

array([[[[6.54009511e-01, 8.32667268e-17],
         [1.17961196e-16, 1.87521981e-01]],

        [[1.11022302e-16, 6.45249427e-01],
         [1.87521981e-01, 1.11022302e-16]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [6.45249427e-01, 1.38777878e-16]],

        [[1.87521981e-01, 5.55111512e-17],
         [2.22044605e-16, 6.78136184e-01]]]])

In [13]:
h1Diagrams = h1Tensor.getAllDiagrams(vacuum)
h2Diagrams = h2Tensor.getAllDiagrams(vacuum)

In [14]:
for d in h1Diagrams:
    print(d.array)
    print(d.getOperator(spinFree=True))

[[-1.18985062]]
-1.0 * a_{h_{1}\beta}a^{h_{0}\beta}
 + -1.0 * a_{h_{1}\alpha}a^{h_{0}\alpha}
[[2.60021255e-17]]
1.0 * a^{h_{0}\beta}a_{p_{0}\beta}
 + 1.0 * a^{h_{0}\alpha}a_{p_{0}\alpha}
[[-4.71423569e-17]]
1.0 * a^{p_{0}\beta}a_{h_{0}\beta}
 + 1.0 * a^{p_{0}\alpha}a_{h_{0}\alpha}
[[-0.5337491]]
1.0 * a^{p_{0}\beta}a_{p_{1}\beta}
 + 1.0 * a^{p_{0}\alpha}a_{p_{1}\alpha}


In [15]:
for d in h2Diagrams:
    print(d.array)
    print(d.getOperator(spinFree=True))

[[[[0.65400951]]]]
1.0 * a_{h_{3}\beta}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}
[[[[8.32667268e-17]]]]
-1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\alpha}
 + -1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{0}\alpha}
[[[[1.17961196e-16]]]]
1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{0}\beta}
 + 1.0 * a_{h_{2}\beta}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\alpha}
 + 1.0 * a_{h_{2}\alpha}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\beta}
 + 1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{0}\alpha}
[[[[0.18752198]]]]
1.0 * a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\beta}
 + 1.0 * a^{h_{0}\alpha}a^{h_{1}\beta}a_{p

In [16]:
HamiltonianTensor = sum(h1Diagrams) + 0.5 * sum(h2Diagrams)

In [17]:
print(HamiltonianTensor)

1.0 * f_{h_{0}}^{h_{1}}
 + 1.0 * f_{h_{0}}^{p_{0}}
 + 1.0 * f_{p_{0}}^{h_{0}}
 + 1.0 * f_{p_{0}}^{p_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}p_{0}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}h_{2}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}h_{0}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}p_{3}}


In [18]:
t2Tensor.array = np.zeros((1,1,1,1))

In [79]:
#t2Tensor.array[0,0,0,0] = -0.275333314977893
t2Tensor.array[0,0,0,0] = -0.13766666

In [82]:
print(GeneralisedWick.evaluateWick(HamiltonianTensor * 0.25 * t2Tensor, spinFree=True))

0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}


In [81]:
GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(HamiltonianTensor * 0.25 * t2Tensor, spinFree=True))

-0.01290776239372985

In [83]:
GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(BCHSimilarityTransform(HamiltonianTensor, 0.25 * t2Tensor, 2), spinFree=True))

-0.01290776239372985

In [23]:
projectionDoubles = projectionManifold(2)

In [24]:
projectionDoubles.array = np.ones_like(t2Tensor.array)

In [84]:
GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * BCHSimilarityTransform(HamiltonianTensor, 0.25 * t2Tensor, 2), spinFree=True, normalOrderedParts=False))

0.3129146200075132

In [26]:
ConstantAmplitude = GeneralisedWick.evaluateWick(projectionDoubles * 0.5 * sum(h2Diagrams), spinFree=True)
LinearAmplitudeFock = GeneralisedWick.evaluateWick(projectionDoubles * sum(h1Diagrams) * 0.25 * t2Tensor, spinFree=True)
LinearAmplitudeFluctuation = GeneralisedWick.evaluateWick(projectionDoubles * 0.5 * sum(h2Diagrams) * 0.25 * t2Tensor, spinFree=True)
QuadraticAmplitude = GeneralisedWick.evaluateWick(projectionDoubles * 0.5 * sum(h2Diagrams) * 0.25 * t2Tensor * 0.25 * t2Tensor, spinFree=True)

In [86]:
GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * sum(h1Diagrams) * 0.25 * t2Tensor, spinFree=True) + (-1.) * GeneralisedWick.evaluateWick(projectionDoubles * 0.25 * t2Tensor * sum(h1Diagrams), spinFree=True))

-0.18064660891784273

In [88]:
GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * 0.5 * sum(h2Diagrams) * 0.25 * t2Tensor, spinFree=True) + GeneralisedWick.evaluateWick(projectionDoubles * 0.25 * t2Tensor * 0.5 * sum(h2Diagrams), spinFree=True))

0.12029423565675593

In [89]:
0.5 * GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * 0.5 * sum(h2Diagrams) * 0.25 * t2Tensor * 0.25 * t2Tensor, spinFree=True)) - GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * 0.25 * t2Tensor * 0.5 * sum(h2Diagrams) * 0.25 * t2Tensor, spinFree=True)) + 0.5 * GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * 0.25 * t2Tensor * 0.25 * t2Tensor * 0.5 * sum(h2Diagrams), spinFree=True))

-0.0017769685368183939

In [30]:
GeneralisedWick.contractTensorSum(ConstantAmplitude) + GeneralisedWick.contractTensorSum(LinearAmplitudeFock) + GeneralisedWick.contractTensorSum(LinearAmplitudeFluctuation) + GeneralisedWick.contractTensorSum(QuadraticAmplitude) + GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(HamiltonianTensor * 0.25 * t2Tensor, spinFree=True)) * t2Tensor.array[0,0,0,0]

0.2614470913728871

In [31]:
GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(HamiltonianTensor * 0.25 * t2Tensor, spinFree=True))# * t2Tensor.array[0,0,0,0]

-0.025815524316581975

In [32]:
GeneralisedWick.contractTensorSum(QuadraticAmplitude)

-8.673617379884035e-19

In [33]:
print(ConstantAmplitude)

2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{0}h_{1}}


In [34]:
GeneralisedWick.contractTensorSum(ConstantAmplitude)

0.3750439618054172

In [35]:
print(LinearAmplitudeFock)

0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{2}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}{t_{2}}_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.5 

In [36]:
GeneralisedWick.contractTensorSum(LinearAmplitudeFock)

-0.3612932112456614

In [37]:
print(LinearAmplitudeFluctuation)

-0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{3}h_{2}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{3}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}p_{1}}^{h_{0}p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{2}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}p_{1}}^{h_{0}p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{2}

In [38]:
GeneralisedWick.contractTensorSum(LinearAmplitudeFluctuation)

0.24058846692515437

In [39]:
print(HamiltonianTensor.getOperator(spinFree=True))

-1.0 * a_{h_{1}\beta}a^{h_{0}\beta}
 + -1.0 * a_{h_{1}\alpha}a^{h_{0}\alpha}
 + 1.0 * a^{h_{0}\beta}a_{p_{0}\beta}
 + 1.0 * a^{h_{0}\alpha}a_{p_{0}\alpha}
 + 1.0 * a^{p_{0}\beta}a_{h_{0}\beta}
 + 1.0 * a^{p_{0}\alpha}a_{h_{0}\alpha}
 + 1.0 * a^{p_{0}\beta}a_{p_{1}\beta}
 + 1.0 * a^{p_{0}\alpha}a_{p_{1}\alpha}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}
 + 0.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\alpha}
 + 0.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{0}\alpha}
 + 1.0 * a_{h_{2}\beta}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\alpha}
 + 1.0 * a_{h_{2}\alpha}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\beta}
 + 1.0 

In [40]:
-0.5 * GeneralisedWick.contractTensorSum(GeneralisedWick.evaluateWick(projectionDoubles * 0.25 * t2Tensor * 0.25 * t2Tensor * HamiltonianTensor * 0.25 * t2Tensor, spinFree=True))

-0.0

Old Code version

In [41]:
def spinFreeSingleExcitation(p, q):
    summandList = []
    for spin in range(2):
        summandList.append(Wick.operatorProduct([Wick.basicOperator(p,1,spin), Wick.basicOperator(q,0,spin)]))
    return Wick.operatorSum(summandList)

In [42]:
def spinFreeDoubleExcitation(p, q, r, s):
    summandList = []
    for spin1 in range(2):
        for spin2 in range(2):
            summandList.append(Wick.operatorProduct([Wick.basicOperator(p,1,spin2), Wick.basicOperator(q,1,spin1), Wick.basicOperator(s,0,spin1), Wick.basicOperator(r,0,spin2)]))
    return Wick.operatorSum(summandList)

In [43]:
h1Operator = Wick.operatorSum([])
for p in range(Norbs):
    for q in range(Norbs):
        h1Operator = h1Operator + h1[p, q] * spinFreeSingleExcitation(p, q)

In [44]:
h2Operator = Wick.operatorSum([])
h2Array = np.zeros((Norbs,Norbs,Norbs,Norbs))
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(p + 1):
            for s in range(q + 1):
                x = int(p + Norbs * r - 0.5 * r * (r + 1))
                y = int(q + Norbs * s - 0.5 * s * (s + 1))
                print(p, q, r, s)
                print(x, y)
                if p == r and q == s:
                    h2Operator = h2Operator + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)
                    h2Array[p,q,r,s] += 0.5 * eri[x, y]
                else:
                    h2Operator = h2Operator + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s) + 0.5 * np.conjugate(eri[x, y]) * spinFreeDoubleExcitation(r, s, p, q)
                    h2Array[p,q,r,s] += 0.5 * eri[x, y]
                    h2Array[r,s,p,q] += 0.5 * np.conjugate(eri[x, y])

0 0 0 0
0 0
0 1 0 0
0 1
0 1 0 1
0 2
1 0 0 0
1 0
1 0 1 0
2 0
1 1 0 0
1 1
1 1 0 1
1 2
1 1 1 0
2 1
1 1 1 1
2 2


In [45]:
h2Array

array([[[[3.27004756e-01, 4.16333634e-17],
         [5.20417043e-17, 9.37609905e-02]],

        [[4.16333634e-17, 3.22624713e-01],
         [0.00000000e+00, 2.77555756e-17]]],


       [[[5.20417043e-17, 0.00000000e+00],
         [3.22624713e-01, 6.93889390e-17]],

        [[9.37609905e-02, 2.77555756e-17],
         [6.93889390e-17, 3.39068092e-01]]]])

In [46]:
doublesAmplitudes = np.array([[[[-0.275333314977893]]]])
clusterDoubles = Wick.operatorSum([])
for a in range(Nocc):
    for b in range(Nocc):
        for k in range(Norbs - Nocc):
            r = k + Nocc
            for l in range(Norbs - Nocc):
                s = l + Nocc
                clusterDoubles = clusterDoubles + doublesAmplitudes[a,b,k,l] * spinFreeDoubleExcitation(r, s, a, b)
clusterDoubles.checkNilpotency()

In [47]:
projectionDoublesOperator = spinFreeDoubleExcitation(1,1,0,0).conjugate()

In [48]:
print(projectionDoublesOperator)

1.0 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{1\alpha}
 + 1.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 1.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 1.0 * a^{0\beta}a^{0\beta}a_{1\beta}a_{1\beta}


In [49]:
ConstantAmplitudeOld = Wick.vacuumExpectationValue(projectionDoublesOperator * h2Operator, vacuum, printing=True)

0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


In [50]:
print(ConstantAmplitudeOld)

0.37504396180541716


In [51]:
LinearAmplitudeFockOld = Wick.vacuumExpectationValue(projectionDoublesOperator * h1Operator * 0.25 * clusterDoubles, vacuum, printing=True) - Wick.vacuumExpectationValue(projectionDoublesOperator * 0.25 * clusterDoubles * h1Operator, vacuum, printing=True)

0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.03673972738168411 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.03673972738168411 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.03673972738168411 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.03673972738168411 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a_{1\beta}a^{1\beta}a^{1\alph

In [52]:
Wick.vacuumExpectationValue(projectionDoublesOperator * 0.25 * clusterDoubles * h1Operator, vacuum, printing=True)

0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\alpha}a_{0\alpha}
0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\beta}a_{0\beta}
0.08190137878739176 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
0.08190137878739176 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
0.08190137878739176 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\alpha}a_{0\alpha}
0.08190137878739176 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\be

0.6552110302991342

In [53]:
print(LinearAmplitudeFockOld)

-0.3612932112456613


In [94]:
print(Wick.normalOrder(projectionDoublesOperator, vacuum))
print(Wick.normalOrder(h2Operator, vacuum))
print(Wick.normalOrder(clusterDoubles, vacuum))

1.0 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{1\alpha}
 + 1.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 1.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 1.0 * a^{0\beta}a^{0\beta}a_{1\beta}a_{1\beta}
0.0
 + 0.3270047556780619 * a_{0\alpha}a_{0\beta}a^{0\beta}a^{0\alpha}
 + -4.163336342344337e-17 * a_{0\alpha}a^{0\alpha}a^{0\beta}a_{1\beta}
 + 5.204170427930421e-17 * a_{0\alpha}a^{0\beta}a^{0\alpha}a_{1\beta}
 + 0.3270047556780619 * a_{0\beta}a_{0\alpha}a^{0\alpha}a^{0\beta}
 + 0.3270047556780619 * a_{0\beta}a_{0\beta}a^{0\beta}a^{0\beta}
 + 5.204170427930421e-17 * a_{0\beta}a^{0\alpha}a^{0\beta}a_{1\alpha}
 + -4.163336342344337e-17 * a_{0\beta}a^{0\beta}a^{0\alpha}a_{1\alpha}
 + 1.0408340855860843e-17 * a_{0\beta}a^{0\beta}a^{0\beta}a_{1\beta}
 + 1.0408340855860843e-17 * a^{1\alpha}a_{0\alpha}a_{0\alpha}a^{0\alpha}
 + -4.163336342344337e-17 * a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\beta}
 + -0.6452494265884846 * a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}
 + 5.204170427930421e-17 * 

In [97]:
print(projectionDoublesOperator * h2Operator * 0.25 * clusterDoubles)

0.0
 + -0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{1\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{0\beta}a^{0\alpha}a_{1

In [ ]:
print(Wick.normalOrder(projectionDoublesOperator, vacuum) * Wick.normalOrder(h2Operator, vacuum) * 0.25 * Wick.normalOrder(clusterDoubles, vacuum))

In [98]:
Wick.vacuumExpectationValue(Wick.normalOrder(projectionDoublesOperator, vacuum) * Wick.normalOrder(h2Operator, vacuum) * 0.25 * Wick.normalOrder(clusterDoubles, vacuum), vacuum, printing=True)

-0.02250882584859419 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a_{0\alpha}a_{0\beta}a^{0\beta}a^{0\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.02250882584859419 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a_{0\alpha}a_{0\beta}a^{0\beta}a^{0\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.02250882584859419 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a_{0\beta}a_{0\alpha}a^{0\alpha}a^{0\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.02250882584859419 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a_{0\beta}a_{0\alpha}a^{0\alpha}a^{0\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.044414665902548014 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.044414665902548014 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.044414665902548014 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{0\beta}a^{0\beta}

0.34385056419148197

In [100]:
Wick.vacuumExpectationValue(Wick.normalOrder(projectionDoublesOperator, vacuum) * 0.25 * Wick.normalOrder(clusterDoubles, vacuum) * Wick.normalOrder(h2Operator, vacuum), vacuum, printing=True)

0.0

In [55]:
LinearAmplitudeFluctuationOld = Wick.vacuumExpectationValue(projectionDoublesOperator * h2Operator * 0.25 * clusterDoubles, vacuum, printing=True)- Wick.vacuumExpectationValue(projectionDoublesOperator * 0.25 * clusterDoubles * h2Operator, vacuum, printing=True)

-0.02333918543256657 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.02333918543256657 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.02333918543256657 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.02333918543256657 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.02333918543256657 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.02333918543256657 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.02333918543256657 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{

In [56]:
print(LinearAmplitudeFluctuationOld)

-0.006642876671779069


In [57]:
QuadraticAmplitudeFluctuationOld = 0.5 * Wick.vacuumExpectationValue(projectionDoublesOperator * h2Operator * 0.25 * clusterDoubles * 0.25 * clusterDoubles, vacuum, printing=True) + 0.5 * Wick.vacuumExpectationValue(projectionDoublesOperator * 0.25 * clusterDoubles * 0.25 * clusterDoubles * h2Operator, vacuum, printing=True) - Wick.vacuumExpectationValue(projectionDoublesOperator * 0.25 * clusterDoubles * h2Operator * 0.25 * clusterDoubles, vacuum, printing=True)

0.00044424211799855707 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.00044424211799855707 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.00044424211799855707 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.00044424211799855707 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.00044424211799855707 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.00044424211799855707 * a^{0\

In [58]:
print(QuadraticAmplitudeFluctuationOld)

-0.007107873887976914


In [59]:
ConstantAmplitudeOld + LinearAmplitudeFockOld + LinearAmplitudeFluctuationOld 

0.007107873887976762

In [60]:
Wick.vacuumExpectationValue(BCHSimilarityTransform(h1Operator + h2Operator, 0.25 * clusterDoubles, 2), vacuum, printing=True)

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
-1.1898506186070186 * a^{0\beta}a_{0\beta}
0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
0.3270047556780619 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
-0.006453881079145487 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881079145487 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.006453881079145487 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
-0.006453881079145487 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}


-1.751507250174495

In [61]:
bch = BCHSimilarityTransform(h1Operator + h2Operator, 0.25 * clusterDoubles, 4)
bch.checkNilpotency()
print(bch)

0.0
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}
 + -1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + 0.0
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 0.0
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0
 + 0.0
 + 0.0
 + 0.0
 + 4.163336342344337e-17 * a^{0\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.3226247132942424 * a^{0\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 2.7755575615628914e-17 * a^{0\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0
 + 0.3226247132942424 * a^{0\beta}a^{1\beta}a_{1\beta}a_{0\beta}
 + 0.0
 +

In [62]:
projected = projectionDoublesOperator * BCHSimilarityTransform(h1Operator + h2Operator, 0.25 * clusterDoubles, 2)
amplitudeEq = Wick.vacuumExpectationValue(projected, vacuum, printing=True)
print(amplitudeEq)

0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
-0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_

In [63]:
projected.checkNilpotency()
amplitudeEq = Wick.vacuumExpectationValue(projected, vacuum, printing=True)
print(amplitudeEq)

0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\alpha}a_{0\alpha}
-0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}a^{0\beta}a_{0\beta}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
0.03673972738168411 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
-0.08190137878739176 * a^{0\alpha}a^{0\beta}a_{1\beta}a_

In [64]:
-0.06883332874447325 * 4

-0.275333314977893

In [65]:
print(h2Operator)

0.0
 + 0.0
 + 0.0
 + 0.0
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 2.7755575615628914e-17 * a^{0\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.3270047556780619 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{0\beta}
 + 5.204170427930421e-17 * a^{0\beta}a^{0\alpha}a_{0\alpha}a_{1\beta}
 + 4.163336342344337e-17 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{0\beta}
 + 0.09376099045135429 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 

Difference is in the linear part of the amplitude equations with the 2-body part of the Hamiltonian

In [66]:
print(LinearAmplitudeFluctuation)

-0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{3}h_{2}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{3}}
 + -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{3}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{3}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}p_{1}}^{h_{0}p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{2}h_{1}}
 + 0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}p_{1}}^{h_{0}p_{3}}{t_{2}}_{p_{3}p_{0}}^{h_{2}

In [67]:
print(LinearAmplitudeFluctuation.summandList[0].prefactor, *LinearAmplitudeFluctuation.summandList[0].vertexList)
print(LinearAmplitudeFluctuation.summandList[0].getOperator(spinFree=True))
print(GeneralisedWick.vacuumExpectationValue(LinearAmplitudeFluctuation.summandList[0].getOperator(spinFree=True), vacuum))

-0.25 \Phi_{h_{0}h_{1}}^{p_{0}p_{1}} v_{h_{2}h_{3}}^{h_{1}h_{0}} {t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
1.0 * a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\beta}a_{h_{0}\beta}a_{h_{1}\beta}a^{h_{2}\beta}a^{h_{3}\beta}a^{p_{1}\beta}a^{p_{0}\beta}a_{h_{2}\beta}a_{h_{3}\beta}
 + 1.0 * a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\alpha}a_{h_{0}\beta}a_{h_{1}\beta}a^{h_{2}\beta}a^{h_{3}\beta}a^{p_{1}\beta}a^{p_{0}\beta}a_{h_{2}\beta}a_{h_{3}\beta}
 + 1.0 * a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{1}\alpha}a_{p_{0}\beta}a_{h_{0}\beta}a_{h_{1}\beta}a^{h_{2}\beta}a^{h_{3}\beta}a^{p_{1}\beta}a^{p_{0}\beta}a_{h_{2}\beta}a_{h_{3}\beta}
 + 1.0 * a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{1}\alpha}a_{p_{0}\alpha}a_{h_{0}\beta}a_{h_{1}\beta}a^{h_{2}\beta}a^{h_{3}\beta}a^{p_{1}\beta}a^{p_{0}\beta}a_{h_{2}\beta}a_{h_{3}\beta}
 + 1.0 * a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\beta}a_{h_{0}\beta}a_{h_{1}\alpha}a^{h_{2}\alpha}a^{h_{3}\beta}a^{p_{1}\beta}a^{p_{0}\beta}a_{h_{2}\beta}a_{h_{3}\beta}
 + 1

In [68]:
for term in LinearAmplitudeFluctuation.summandList:
    print(GeneralisedWick.getContractedArray(term), term)

(0.04501765169718838, [], []) -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
(-0.09003530339437676, [], []) 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{3}}
(-0.09003530339437676, [], []) 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{3}h_{2}}
(0.04501765169718838, [], []) -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{1}h_{0}}{t_{2}}_{p_{0}p_{1}}^{h_{2}h_{3}}
(-0.09003530339437676, [], []) 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{3}h_{2}}
(0.04501765169718838, [], []) -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{2}h_{3}}
(0.04501765169718838, [], []) -0.25 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{3}h_{2}}
(-0.09003530339437676, [], []) 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}h_{3}}^{h_{0}h_{1}}{t_{2}

In [69]:
print(LinearAmplitudeFluctuation.summandList[8])
print(LinearAmplitudeFluctuation.summandList[-24])

-0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{h_{2}p_{1}}^{h_{0}p_{3}}{t_{2}}_{p_{0}p_{3}}^{h_{2}h_{1}}
-0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}h_{2}}^{p_{3}h_{0}}{t_{2}}_{p_{0}p_{3}}^{h_{2}h_{1}}


In [70]:
print(LinearAmplitudeFluctuation.summandList[8].vertexList[1].getOperator(spinFree=True, normalOrdered=True))

-1.0 * a^{p_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\beta}
 + -1.0 * a^{p_{1}\beta}a_{h_{0}\alpha}a^{h_{2}\alpha}a_{p_{3}\beta}
 + -1.0 * a^{p_{1}\alpha}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\alpha}
 + -1.0 * a^{p_{1}\alpha}a_{h_{0}\alpha}a^{h_{2}\alpha}a_{p_{3}\alpha}


In [71]:
print(LinearAmplitudeFluctuation.summandList[-24].vertexList[1].getOperator(spinFree=True, normalOrdered=True))

-1.0 * a^{p_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\beta}
 + -1.0 * a^{p_{1}\alpha}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\alpha}
 + -1.0 * a^{p_{1}\beta}a_{h_{0}\alpha}a^{h_{2}\alpha}a_{p_{3}\beta}
 + -1.0 * a^{p_{1}\alpha}a_{h_{0}\alpha}a^{h_{2}\alpha}a_{p_{3}\alpha}


In [72]:
print(LinearAmplitudeFluctuation.summandList[8].getOperator(spinFree=True))
print(LinearAmplitudeFluctuation.summandList[-24].getOperator(spinFree=True))

-1.0 * a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\beta}a^{p_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\beta}a^{p_{0}\beta}a^{p_{3}\beta}a_{h_{1}\beta}a_{h_{2}\beta}
 + -1.0 * a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\alpha}a^{p_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\beta}a^{p_{0}\beta}a^{p_{3}\beta}a_{h_{1}\beta}a_{h_{2}\beta}
 + -1.0 * a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{1}\alpha}a_{p_{0}\beta}a^{p_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\beta}a^{p_{0}\beta}a^{p_{3}\beta}a_{h_{1}\beta}a_{h_{2}\beta}
 + -1.0 * a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{1}\alpha}a_{p_{0}\alpha}a^{p_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a_{p_{3}\beta}a^{p_{0}\beta}a^{p_{3}\beta}a_{h_{1}\beta}a_{h_{2}\beta}
 + -1.0 * a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\beta}a^{p_{1}\beta}a_{h_{0}\alpha}a^{h_{2}\alpha}a_{p_{3}\beta}a^{p_{0}\beta}a^{p_{3}\beta}a_{h_{1}\beta}a_{h_{2}\beta}
 + -1.0 * a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\alpha}a^{p_{1}\beta}a_{h_{0}\alpha}a

In [73]:
print(GeneralisedWick.getContractedArray(LinearAmplitudeFluctuation.summandList[8]))
print(GeneralisedWick.getContractedArray(LinearAmplitudeFluctuation.summandList[-24]))

(0.08882933180509607, [], [])
(0.088829331805096, [], [])


In [74]:
total = 0
for term in LinearAmplitudeFluctuation.summandList[:8] + LinearAmplitudeFluctuation.summandList[-8:]:
#    print(GeneralisedWick.getContractedArray(term))
    total += GeneralisedWick.getContractedArray(term)[0]
print(total)

-0.36678409024928604


In [75]:
LinearAmplitudeFluctuationOld

-0.006642876671779069

In [76]:
print(clusterDoubles)

0.0
 + -0.275333314977893 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.275333314977893 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0


In [77]:
-0.275333314977893 * 0.25 * h2Array[0,0,0,0]

-0.02250882584859419

In [78]:
print(projectionDoublesOperator * h2Operator * 0.25 * clusterDoubles)
print(projectionDoublesOperator * 0.25 * clusterDoubles * h2Operator)

0.0
 + -0.0 * a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + -0.0 * a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\beta}a^{0\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{0\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\alpha}a^{1\beta}a_{1\beta}a_{1\alpha}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{0\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{0\alpha}a_{1\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{0\beta}
 + 0.0 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}a^{1\beta}a^{1\alpha}a_{1\alpha}a_{1\beta}
 + 0.0 * a^{0\beta}a^{0\alpha}a_{1